In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep 
import requests
import pandas as pd
import numpy as np

In [2]:
def get_links(url, start_year = 'NG', end_year = 'NG', manufacturer_name = 'any', login = 'login', password = 'password', start_price = 'NG', end_price = 'NG', fuel_type = 'NG'):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu') 

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url=url)
    driver.maximize_window()
    wait = WebDriverWait(driver, 10)  
    sleep(2)
    cookie = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[3]/div/button')))
    cookie.click()
    if login != 'login' and password != 'password':
        login_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[1]/header/div[2]/div/div/div[2]/div[2]')))
        if "შესვლა" in login_button.text:
            login_button.click()

            email_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/form/div[1]/div[1]/div/input')))
            email_input.send_keys(login) # "kordzaxianika@gmail.com"

            password_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/form/div[1]/div[2]/div/input')))
            password_input.send_keys(password) # "Seleniumtest01"

            login_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/form/div[3]/button')
            login_button.click()
    
    actions = ActionChains(driver) 

    if manufacturer_name != 'any':
        manufacturer_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[1]')))
        manufacturer_button.click() 
        sleep(5)
        models = driver.find_elements(By.XPATH, '//label[@class="flex relative cursor-pointer px-[24px] py-[10px] mb-0 "]//span[@class="flex text-[14px] md:text-[12px] xl:text-[14px] "]')
        for model in models:                     
            if model.text == manufacturer_name:
                actions.move_to_element(model).perform()  
                sleep(1) 
                model.click()
                break

    my_auto = driver.find_element(By.CLASS_NAME, 'd-flex.icon-h-32px.icon-h-sm-40px') 
    actions.move_to_element(my_auto).perform()  
    my_auto.click()
    sleep(10)
    if start_year != 'NG' or end_year != 'NG':
        years = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[5]')))
        ActionChains(driver).move_to_element(years).perform()
        years.click()
        sleep(3)
        start_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[5]/div[2]/div[1]/div[1]/div[1]/div/div/input')))
        start_input.send_keys(str(start_year))
        
        end_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[5]/div[2]/div[1]/div[1]/div[2]/div/div/input')))
        end_input.send_keys(str(end_year))
    sleep(5)
    if start_price != 'NG' or end_price != 'NG':
        prices = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[6]')))
        ActionChains(driver).move_to_element(prices).perform()
        prices.click()
        sleep(3)
        start_price_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[6]/div[2]/div[1]/div[2]/div[1]/div/div/input')))
        start_price_input.send_keys(str(start_price))
        
        end_price_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[6]/div[2]/div[1]/div[2]/div[2]/div/div/input')))
        end_price_input.send_keys(str(end_price))
    
    if fuel_type != 'NG':
        fuels = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[1]/div[2]/div[2]/div/div[7]')))
        ActionChains(driver).move_to_element(fuels).perform()
        fuels.click()
        sleep(2)
        fuel_buttons = driver.find_elements(By.XPATH, "//label[contains(@class, 'cursor-pointer')]")
 
        for button in fuel_buttons:
            if fuel_type == button.text:
                actions = ActionChains(driver)
                actions.move_to_element(button).perform()
                button.click()
                break
    buttons = driver.find_elements(By.TAG_NAME, "button") 
    for button in buttons:
        if "ძებნა" in button.text:
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()
            button.click()
            break
    sleep(5)
    list_link = []
    n = 1
    next_button = True
    while next_button:
        if n > 1:
            nxt_buttons = driver.find_elements(By.TAG_NAME, "li")
            found = False
            for button in nxt_buttons:
                if str(n) == button.text:
                    actions = ActionChains(driver)
                    actions.move_to_element(button).perform()
                    button.click()
                    found = True
                    break
            if not found:
                next_button = False
            sleep(3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        links = soup.find_all('a', attrs={'class': 'w-full md:w-auto'})
        for link in links:
            href = link.get('href')
            list_link.append('https://www.myauto.ge' + href)
        n += 1
    sleep(1) 
    if driver:
        driver.quit()
    return list_link

In [ ]:
links = get_links(url='https://www.myauto.ge/ka',  start_year = 2000, end_year=2000, manufacturer_name = 'Audi')

In [13]:
def read_text(data_list, link, index):
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('--disable-gpu') 

    # Use WebDriver manager to handle ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    try:
        driver.get(link)
        wait = WebDriverWait(driver, 10)

        model = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="sticky-bottom-boundary"]/div[1]/div[1]/div[1]/div[2]/span[1]')))
        
        data_list['Index'].append(index)
        data_list['Link'].append(link)
        data_list['Model'].append(model.text)
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        driver.quit()
    return data_list

In [14]:
def read_links():
    global links
    data_list = {}
    data_list['Index'] = []
    data_list['Link'] = []
    data_list['Model'] = []
    n = 1
    #links = get_links(url='https://www.myauto.ge/ka',  start_year = 2000, end_year=2000, manufacturer_name = 'Audi')
    index = 1
    for link in links:
        sleep(1)
        data_list = read_text(data_list, link, index)
        index = index + 1
        print(n)
        n += 1
    df = pd.DataFrame(data_list)
    return df

In [ ]:
%%time
read_links()